In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import datetime as dt
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func,inspect

from flask import Flask, jsonify

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# SQL Connection
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)

In [ ]:
# Use the Inspector to explore the database and print the table names
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
columns = inspector.get_columns('station')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
data = engine.execute("select * from measurement limit 5").fetchall()
for x in data:
    print(x)

In [ ]:
# Reflect Database into ORM class
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Precipitation Analysis

In [ ]:
# Last Date
last_date = session.query(Measurement.date).\
    order_by(Measurement.date.desc()).first().date
last_date

In [ ]:
past12mth_date = dt.datetime.strptime(last_date, '%Y-%m-%d') - dt.timedelta(days=365)
past12mth_date

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
prec_results = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date >= past12mth_date).\
    group_by(Measurement.date).all()         
print(prec_results)


In [ ]:
#Pandas DataFrame for data and precipitation scores for the last 12 months
prec_df = pd.DataFrame(prec_results, columns=['Date', 'Precipitation'])
prec_df = prec_df.set_index('Date')
prec_df.head()

In [ ]:
#Plot the results using the DataFrame `plot` method
prec_df.plot(kind="bar", width=3, figsize=(80,8),rot=90)
plt.title(" Last 12 months of precipitation data")
plt.ylabel("Precipitation (Inches)", size=18)
plt.xlabel("Date", size=18)
plt.grid(True)
plt.show()
plt.tight_layout()
plt.savefig("Images/Precipitation.png")


In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data

prec_df.describe()

### Station Analysis

In [ ]:
data = engine.execute("select * from Station").fetchall()
data

In [ ]:
# Design a query to show how many stations are available in this dataset?
station_ct = session.query(Station.station).count()
print(station_ct) 

In [ ]:
station_nm = session.query(Station.name).all()
station_nm

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
active_station = session.query(Measurement.station, func.count(Measurement.station)).\
            group_by(Measurement.station).\
            order_by(func.count(Measurement.station).desc()).all()
active_station    



In [ ]:
most_active_st = active_station[0][0]
most_active_st

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?
temp_station = session.query(Measurement.station,
                               func.min(Measurement.tobs),
                               func.max(Measurement.tobs),  
                               func.avg(Measurement.tobs)).\
            filter(Measurement.station == most_active_st).all()
temp_station

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
highT_station = session.query(Measurement.station, Measurement.tobs).\
                filter(Measurement.station == most_active_st).\
                filter(Measurement.date >= past12mth_date).all()
   
highT_station_df = pd.DataFrame(highT_station, columns=['Station', 'Tobs'])
highT_station_df = highT_station_df.set_index('Station')
highT_station_df.head()

In [ ]:
highT_station_df.hist(bins=12, figsize=(12,8),label = 'Tobs')
plt.grid(True)
plt.title("Highest Temperature Obs for " + most_active_st,fontsize=16)
plt.xlabel("Temperature Obs", fontsize=16)
plt.savefig("Images/HighestStationTemps.png")
plt.legend(loc='upper right')
plt.show
plt.savefig("Images/hightemperature.png")

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    temp_results = session.query(func.min(Measurement.tobs),
                              func.avg(Measurement.tobs),
                              func.max(Measurement.tobs)).\
               filter(Measurement.date >= start_date).\
               filter(Measurement.date <= end_date).all() 
    return temp_results
                                 
calc_temps('2017-01-01', '2017-08-00')

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


# Step 2 - Climate App

In [ ]:
app = Flask(__name__)

In [ ]:
@app.route("/")
def welcome1():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation1<br/>"
        f"/api/v1.0/<start_date>/<end_date>"
    )

In [ ]:
@app.route("/api/v1.0/precipitation1")
def precipitation1():
    last_date = session.query(Measurement.date).\
    order_by(Measurement.date.desc()).first().date
    past12mth_date = dt.datetime.strptime(last_date, '%Y-%m-%d') - dt.timedelta(days=365)


    tobs_results = session.query(Measurement.date, Measurement.tobs).\
        filter(Measurement.date >= past12mth_date).\
        group_by(Measurement.date).all()  

    
    temp_dict = [{element[0]:element[1]} for element in tobs_results]
        
    return  jsonify(temp_dict) 
    

In [77]:
@app.route("/api/v1.0/stations")
def stations():
    station_nm = session.query(Station.station,Station.name).all()
    return jsonify(station_nm)

AssertionError: View function mapping is overwriting an existing endpoint function: stations

In [ ]:
@app.route("/api/v1.0/<start_date>/<end_date>")
def start_end1(start_date,end_date):
    temp_results = session.query(func.min(Measurement.tobs),
                            func.avg(Measurement.tobs),
                            func.max(Measurement.tobs)).\
            filter(Measurement.date >= start_date).\
            filter(Measurement.date <= end_date).all() 
    return jsonify(temp_results)
     